In [1]:
import json
import requests
import urllib.request
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.DataFrame()
index = 0
last_date = ''

# 수정 변수들
coinName = "XRP"
to_date = '2021-05-01'
num = 30
scale = '1'
if num == 0:
    type = 'days'
else:
    type = 'minutes/{}'.format(num)


def get_coin_data_url(coinName, type, scale, cnt=400) :
    addr = 'https://crix-api-endpoint.upbit.com/v1/crix/candles/'
    if type == 'minutes' :
        basic_url = addr + type + '/' + scale + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)
    else :
        basic_url = addr + type + '/' + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)

    return basic_url

basic_url = get_coin_data_url(coinName,type,scale)
url = basic_url

def get_upbit_data(url, last_date, to_date) :
    global df
    global index

    fail2GetData = False
    response = ''
    while True:
        try :
            response = requests.get(url)
        except Exception as e:
            time.sleep(20)
            continue
        if str(response) == '<Response [200]>':
            break
        time.sleep(10)
    if ( fail2GetData )  :
        exit()

    data = response.json()

    if index == 0:
        df = pd.DataFrame(data)
        date = ''
        date = data[len(data)-1]['candleDateTime']
        index +=1

    else:
        df_for = pd.DataFrame(data)
        df = df.append(df_for,ignore_index=True)
        date = ''
        date = data[len(data)-1]['candleDateTime']


    return date




In [2]:

while (1) :
    last_date = get_upbit_data(url, last_date, to_date)
    tmp1 = last_date.split('T')
    if tmp1[0]  < to_date :
        break
    tmp2 = tmp1[1].split('+')
    target_date = tmp1[0] + ' ' + tmp2[0]
    url = basic_url + '&to=' + target_date    #to=2019-11-27 04:01:00
    time.sleep(2)

idx_todate = df.index[(df.candleDateTimeKst == '{}T00:00:00+09:00'.format(to_date))].tolist()[0]+1
idx_todate

df = df.drop(df.index[idx_todate:])
df

,code,candleDateTime,candleDateTimeKst,openingPrice,highPrice,lowPrice,tradePrice,candleAccTradeVolume,candleAccTradePrice,timestamp,unit
0,CRIX.UPBIT.KRW-XRP,2021-05-13T07:00:00+00:00,2021-05-13T16:00:00+09:00,1715.0,1725.0,1705.0,1715.0,9.479592e+06,1.626809e+10,1620890666675,30
1,CRIX.UPBIT.KRW-XRP,2021-05-13T06:30:00+00:00,2021-05-13T15:30:00+09:00,1705.0,1715.0,1690.0,1715.0,1.074755e+07,1.830670e+10,1620889200251,30
2,CRIX.UPBIT.KRW-XRP,2021-05-13T06:00:00+00:00,2021-05-13T15:00:00+09:00,1720.0,1720.0,1700.0,1705.0,1.027039e+07,1.756576e+10,1620887399856,30
3,CRIX.UPBIT.KRW-XRP,2021-05-13T05:30:00+00:00,2021-05-13T14:30:00+09:00,1710.0,1720.0,1695.0,1720.0,1.464163e+07,2.501423e+10,1620885599919,30
4,CRIX.UPBIT.KRW-XRP,2021-05-13T05:00:00+00:00,2021-05-13T14:00:00+09:00,1690.0,1715.0,1685.0,1705.0,1.550158e+07,2.636217e+10,1620883800231,30
...,...,...,...,...,...,...,...,...,...,...,...
604,CRIX.UPBIT.KRW-XRP,2021-04-30T17:00:00+00:00,2021-05-01T02:00:00+09:00,1865.0,1880.0,1840.0,1850.0,1.152331e+07,2.141242e+10,1619803800455,30
605,CRIX.UPBIT.KRW-XRP,2021-04-30T16:30:00+00:00,2021-05-01T01:30:00+09:00,1865.0,1895.0,1860.0,1865.0,1.559364e+07,2.931570e+10,1619802000624,30
606,CRIX.UPBIT.KRW-XRP,2021-04-30T16:00:00+00:00,2021-05-01T01:00:00+09:00,1860.0,1900.0,1855.0,1870.0,3.212745e+07,6.050358e+10,1619800200875,30
607,CRIX.UPBIT.KRW-XRP,2021-04-30T15:30:00+00:00,2021-05-01T00:30:00+09:00,1855.0,1865.0,1840.0,1855.0,2.543442e+07,4.711179e+10,1619798400428,30


In [3]:
df['Date'] = 0
df['hour'] = 0
df['minute'] = 0

In [4]:
df['Date'][0] = df['candleDateTimeKst'][0].split('T')
df['Date'][0][0]


'2021-05-13'

In [5]:
for i in range(len(df)):
    df['Date'] [i] = df['candleDateTimeKst'][i].split('T')
    df['hour'] [i] = df['Date'][i][1].split('+')[0]
    df['minute'] [i] = df['hour'][i].split(':')[1]
    df['hour'] [i] = df['hour'][i].split(':')[0]
    df['Date'] [i] = df['Date'][i][0]


In [6]:
df

,code,candleDateTime,candleDateTimeKst,openingPrice,highPrice,lowPrice,tradePrice,candleAccTradeVolume,candleAccTradePrice,timestamp,unit,Date,hour,minute
0,CRIX.UPBIT.KRW-XRP,2021-05-13T07:00:00+00:00,2021-05-13T16:00:00+09:00,1715.0,1725.0,1705.0,1715.0,9.479592e+06,1.626809e+10,1620890666675,30,2021-05-13,16,0
1,CRIX.UPBIT.KRW-XRP,2021-05-13T06:30:00+00:00,2021-05-13T15:30:00+09:00,1705.0,1715.0,1690.0,1715.0,1.074755e+07,1.830670e+10,1620889200251,30,2021-05-13,15,30
2,CRIX.UPBIT.KRW-XRP,2021-05-13T06:00:00+00:00,2021-05-13T15:00:00+09:00,1720.0,1720.0,1700.0,1705.0,1.027039e+07,1.756576e+10,1620887399856,30,2021-05-13,15,0
3,CRIX.UPBIT.KRW-XRP,2021-05-13T05:30:00+00:00,2021-05-13T14:30:00+09:00,1710.0,1720.0,1695.0,1720.0,1.464163e+07,2.501423e+10,1620885599919,30,2021-05-13,14,30
4,CRIX.UPBIT.KRW-XRP,2021-05-13T05:00:00+00:00,2021-05-13T14:00:00+09:00,1690.0,1715.0,1685.0,1705.0,1.550158e+07,2.636217e+10,1620883800231,30,2021-05-13,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,CRIX.UPBIT.KRW-XRP,2021-04-30T17:00:00+00:00,2021-05-01T02:00:00+09:00,1865.0,1880.0,1840.0,1850.0,1.152331e+07,2.141242e+10,1619803800455,30,2021-05-01,02,0
605,CRIX.UPBIT.KRW-XRP,2021-04-30T16:30:00+00:00,2021-05-01T01:30:00+09:00,1865.0,1895.0,1860.0,1865.0,1.559364e+07,2.931570e+10,1619802000624,30,2021-05-01,01,30
606,CRIX.UPBIT.KRW-XRP,2021-04-30T16:00:00+00:00,2021-05-01T01:00:00+09:00,1860.0,1900.0,1855.0,1870.0,3.212745e+07,6.050358e+10,1619800200875,30,2021-05-01,01,0
607,CRIX.UPBIT.KRW-XRP,2021-04-30T15:30:00+00:00,2021-05-01T00:30:00+09:00,1855.0,1865.0,1840.0,1855.0,2.543442e+07,4.711179e+10,1619798400428,30,2021-05-01,00,30


In [7]:

drops= ['unit','candleAccTradePrice','code','candleDateTime','timestamp','candleDateTimeKst']
df=df.drop(drops,axis=1)
df=df[['Date','hour','minute','openingPrice','highPrice','lowPrice','tradePrice','candleAccTradeVolume']]
df.head(5)

,Date,hour,minute,openingPrice,highPrice,lowPrice,tradePrice,candleAccTradeVolume
0,2021-05-13,16,0,1715.0,1725.0,1705.0,1715.0,9.479592e+06
1,2021-05-13,15,30,1705.0,1715.0,1690.0,1715.0,1.074755e+07
2,2021-05-13,15,0,1720.0,1720.0,1700.0,1705.0,1.027039e+07
3,2021-05-13,14,30,1710.0,1720.0,1695.0,1720.0,1.464163e+07
4,2021-05-13,14,0,1690.0,1715.0,1685.0,1705.0,1.550158e+07


In [8]:
df['rsi_ud'] = 0


,Date,hour,minute,openingPrice,highPrice,lowPrice,tradePrice,candleAccTradeVolume,rsi_ud
0,2021-05-13,16,0,1715.0,1725.0,1705.0,1715.0,9.479592e+06,0
1,2021-05-13,15,30,1705.0,1715.0,1690.0,1715.0,1.074755e+07,0
2,2021-05-13,15,0,1720.0,1720.0,1700.0,1705.0,1.027039e+07,0
3,2021-05-13,14,30,1710.0,1720.0,1695.0,1720.0,1.464163e+07,0
4,2021-05-13,14,0,1690.0,1715.0,1685.0,1705.0,1.550158e+07,0
...,...,...,...,...,...,...,...,...,...
604,2021-05-01,02,0,1865.0,1880.0,1840.0,1850.0,1.152331e+07,0
605,2021-05-01,01,30,1865.0,1895.0,1860.0,1865.0,1.559364e+07,0
606,2021-05-01,01,0,1860.0,1900.0,1855.0,1870.0,3.212745e+07,0
607,2021-05-01,00,30,1855.0,1865.0,1840.0,1855.0,2.543442e+07,0
